In [2]:
import sympy as sp
import numpy as np
from bid_space_ascent.symbolic import (
    generate_symbolic_utilities,
    generate_centralized_reduced_utilities,
)

sp.init_printing()

# The case n=2

It turns out that in this case, dynamics are linear (as long as we are in the interior of the domain)

In [16]:
n = 2

f = sp.symarray('f',(2,))
g = sp.symarray('g',(2,))

utility_1, utility_2 = generate_centralized_reduced_utilities(f,g)

print(f"Utility agent 1:{utility_1.simplify()}")
print(f"Utility agent 2:{utility_2.simplify()}")

vectorfield = [
    utility_1.diff(f[0]),
    utility_2.diff(g[0])
]

print(f"Vectorfield V of the dynamics: {[u.simplify() for u in vectorfield]}")

poly_vectorfield = [sp.Poly(v,f[0],g[0]) for v in vectorfield]
A = sp.Matrix([[v.coeff_monomial(f[0]), v.coeff_monomial(g[0])] for v in poly_vectorfield])
c = sp.Matrix([v.coeff_monomial(1) for v in poly_vectorfield])

print(f"V=A(f_0,g_0)^T+c where A={A} and c={c}")

Utility agent 1:-f_0**2/24 + f_0*g_0/12 + f_0/8 + g_0/12 + 1/6
Utility agent 2:f_0*g_0/12 + f_0/12 - g_0**2/24 + g_0/8 + 1/6
Vectorfield V of the dynamics: [-f_0/12 + g_0/12 + 1/8, f_0/12 - g_0/12 + 1/8]
V=A(f_0,g_0)^T+c where A=Matrix([[-1/12, 1/12], [1/12, -1/12]]) and c=Matrix([[1/8], [1/8]])


Hence, a natural choice for a Lyapunov function is L(f_0,g_0) = 1/2|A(f_0,g_0)|^2

In [17]:
print(f"Eigenvalues of A: {A.eigenvals()}")

Eigenvalues of A: {-1/6: 1, 0: 1}


In [19]:
It follows that
$$\langle \nabla L, V \rangle  = (f_0,g_0)A^T\left(A\begin{pmatrix}f_0\\g_0\end{pmatrix} + \begin{pmatrix}1/8\\1/8\end{pmatrix}\right)$$


SyntaxError: invalid syntax (1436944641.py, line 1)

In [4]:
f = sp.symarray('f', (2,))
f_0 = f[0]
f_1 = f[1]
g = sp.symarray('g', (2,))
g_0 = g[0]
g_1 = g[1]
b = sp.Symbol('b', nonnegative = True, real = True)


f_strategy = sp.Piecewise(
    (f_0, b < sp.Rational(1,2)),
    (f_1, b >= sp.Rational(1,2))
)
g_strategy = sp.Piecewise(
    (g_0, b < sp.Rational(1,2)),
    (g_1, b >= sp.Rational(1,2))
)
F_strategy = sp.integrate(f_strategy, (b,0,b))

G_strategy = sp.integrate(g_strategy, (b,0,b))

su1, su2 = generate_symbolic_utilities(f, g)
utility_1 = sp.integrate( (F_strategy-b)*G_strategy*f_strategy, (b,0,1) )
print(utility_1 - su1)
utility_2 = sp.integrate( (G_strategy-b)*F_strategy*g_strategy, (b,0,1) )
print(utility_2 - su2)
print(su1)
print(su2)


reduction_substitution = {f_1: 2 - f_0, g_1: 2 - g_0}
centralization_substitution = {f_0: f_0 + 2, g_0: g_0 + 2}
reduced_utility_1 = utility_1.subs(reduction_substitution).subs(centralization_substitution)
reduced_utility_2 = utility_2.subs(reduction_substitution).subs(centralization_substitution)
print(reduced_utility_1)
cru1, cru2 = generate_centralized_reduced_utilities(f,g)
print(cru1-reduced_utility_1)
print(cru2-reduced_utility_2)

vectorfield = [
    reduced_utility_1.diff(f_0),
    reduced_utility_2.diff(g_0)
]

#print(vectorfield)


0
0
f_0**2*g_0/24 + f_0*f_1*g_0/8 + f_0*f_1*g_1/16 - f_0*g_0/24 + f_1**2*g_0/16 + f_1**2*g_1/24 - 3*f_1*g_0/16 - 5*f_1*g_1/48
f_0*g_0**2/24 + f_0*g_0*g_1/8 - f_0*g_0/24 + f_0*g_1**2/16 - 3*f_0*g_1/16 + f_1*g_0*g_1/16 + f_1*g_1**2/24 - 5*f_1*g_1/48
-f_0**2*g_0/24 + f_0**2*(g_0 + 2)/16 + f_0*g_0*(f_0 + 2)/16 - 5*f_0*g_0/48 - f_0*(f_0 + 2)*(g_0 + 2)/8 + 3*f_0*(g_0 + 2)/16 + (f_0 + 2)**2*(g_0 + 2)/24 - (f_0 + 2)*(g_0 + 2)/24
0
0


In [22]:
n = 2
f = sp.symarray('f',n)
g = sp.symarray('g',n)
variables = np.concatenate([f[:-1],g[:-1]])
#A = sp.symarray('A', (2*(n-1),2*(n-1)))
#C = A.T @ A
cru1, cru2 = generate_centralized_reduced_utilities(f, g)
#solution_guess = (variables.reshape((1,2*(n-1))) @ C @ variables.reshape((2*(n-1),1)))[0,0]
#solution_grad = [solution_guess.diff(v) for v in variables]
vectorfield = [cru1.diff(v) for v in f[:-1]] + [cru2.diff(v) for v in g[:-1]]
print(sp.Poly(cru1,*variables))
print(sp.Poly(vectorfield[0],*variables))

Poly(-1/24*f_0**2 + 1/12*f_0*g_0 + 1/8*f_0 + 1/12*g_0 + 1/6, f_0, g_0, domain='QQ')
Poly(-1/12*f_0 + 1/12*g_0 + 1/8, f_0, g_0, domain='QQ')


In [121]:
a, b  = sp.symbols('a b')
solution_guess = a*f_0**2 - b*f_0*g_0 + a*g_0**2
solution_gradient = [solution_guess.diff(f_0), solution_guess.diff(g_0)]
neg_inner_product = -np.dot(vectorfield, solution_gradient)
print(sp.Poly(vectorfield[0],f_0,g_0))
print(sp.Poly(neg_inner_product, f_0, g_0))

Poly(-1/12*f_0 + 1/12*g_0 + 1/8, f_0, g_0, domain='QQ')
Poly((a/6 + b/12)*f_0**2 + (-a/3 - b/6)*f_0*g_0 + (-a/4 + b/8)*f_0 + (a/6 + b/12)*g_0**2 + (-a/4 + b/8)*g_0, f_0, g_0, domain='QQ[a,b]')


In [111]:
'''utility_calculator = UtilityCalculator(2)

f = sp.symarray("f", (2,))
g = sp.symarray("g", (2,))

utility_1 = utility_calculator.getUtility(f, g)
utility_2 = utility_calculator.getUtility(g, f)
utility_1 = utility_1.subs({f[1]: 2 - f[0], g[1]: 2 - g[0]}).subs(
    {f[0]: f[0] + 2, g[0]: g[0] + 2}
)
utility_2 = utility_2.subs({f[1]: 2 - f[0], g[1]: 2 - g[0]}).subs(
    {f[0]: f[0] + 2, g[0]: g[0] + 2}
)'''

l1, l2 = sp.symbols('l1 l2')
lyapunov_ansatz = l1 * (f[0] ** 2 + g[0] ** 2) + l2 * f[0] * g[0]
ansatz_gradient = [
    lyapunov_ansatz.diff(f[0]).simplify(),
    lyapunov_ansatz.diff(g[0]).simplify(),
]
neg_inner_product = -np.dot(vectorfield, ansatz_gradient)


alpha, beta, gamma, delta, epsilon, phi, mu, nu, xi, omega, omikron = sp.symbols(
    "alpha beta gamma delta epsilon phi mu nu xi omega omikron"
)
chi_0 = nu * (f_0**2 + g_0**2) + alpha * f_0 * g_0
chi_1 = beta * f_0**2 + gamma * g_0**2 + delta * f_0 * g_0
chi_2 = beta * g_0**2 + gamma * f_0**2 + delta * f_0 * g_0
chi_3 = (
    (epsilon * f_0 + phi) ** 2
    + (mu * g_0 + nu) ** 2
    + (xi * f_0 + omikron * g_0 + omega) ** 2
)
chi_4 = (
    (epsilon * g_0 + phi) ** 2
    + (mu * f_0 + nu) ** 2
    + (xi * f_0 + omikron * g_0 + omega) ** 2
)
combination = (
    chi_0 + chi_1 * (f_0 + 2) + chi_2 * (g_0 + 2) + chi_3 * (-f_0) + chi_4 * (-g_0)
)


substitution = {
    beta: 2, #epsilon**2 + xi**2,
    omikron: 1,  # =xi
    xi: 1,
    mu: 1,
    delta: 2,
    gamma: 2,
    l2: -1,
    epsilon: 1,
    nu: sp.Rational(1,4),
    omega: phi,
}

#substitution = {}

print( sp.Poly((neg_inner_product - combination).subs(substitution), f_0, g_0) )


Poly((l1/6 + 4*phi - 49/6)*f_0**2 + (-alpha - l1/3 + 4*phi - 43/6)*f_0*g_0 + (-l1/4 + 2*phi**2 + 3/16)*f_0 + (l1/6 + 4*phi - 49/6)*g_0**2 + (-l1/4 + 2*phi**2 + 3/16)*g_0, f_0, g_0, domain='QQ[l1,alpha,phi]')
